In [9]:
import numpy as np
import pandas as pd

In [14]:
train_df = pd.read_csv('data/train.csv', on_bad_lines='skip')
test_df  = pd.read_csv('data/test.csv', on_bad_lines='skip')

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13177 entries, 0 to 13176
Data columns (total 1 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   	content	label	label_id  13177 non-null  object
dtypes: object(1)
memory usage: 103.1+ KB


In [16]:
train_df.head()
print(train_df.columns)
print(test_df.columns)

Index(['\tcontent\tlabel\tlabel_id'], dtype='object')
Index(['\tcontent\tlabel\tlabel_id'], dtype='object')


In [17]:
trainDf_rowCount = train_df.shape[0]
testDf_rowCount  = test_df.shape[0]

In [18]:
import re

trainTarget = np.empty(trainDf_rowCount, dtype='object')
trainData   = np.empty(trainDf_rowCount, dtype='object')

i = 0
for row in train_df['\tcontent\tlabel\tlabel_id']:
    trainTarget[i] = re.split(r'\t+', row)[-2]
    trainData[i]   = re.split(r'\t+', row)[-3]
    i += 1


In [19]:
testTarget  = np.empty(testDf_rowCount, dtype='object')
testData    = np.empty(testDf_rowCount, dtype='object')

i = 0
for row in test_df['\tcontent\tlabel\tlabel_id']:
    testTarget[i] = re.split(r'\t+', row)[-2]
    testData[i]   = re.split(r'\t+', row)[-3]
    i += 1

In [20]:
# create new dataset
trianDF = pd.DataFrame({'content': trainData, 'category': trainTarget})
testDF  = pd.DataFrame({'content': testData, 'category': testTarget})
testDF.head()

,content,category
0,حسن جوهرچی بازیگر سینما و تلویزیون ایران در گف...,فرهنگی هنری
1,به گزارش گروه بین الملل باشگاه خبرنگاران جوان ...,بین الملل
2,به گزارش خبرنگار فوتبال و فوتسال گروه ورزشی با...,ورزشی
3,به‌ گزارش گروه اقتصادی باشگاه خبرنگاران به نقل...,اقتصادی
4,به گزارش خبرنگار حوزه قرآن و عترت گروه فرهنگی ...,فرهنگی هنری


# Text Preprocessing

In [33]:
import codecs
from hazm import Normalizer

nmz = Normalizer()
stops = "\n".join(
    sorted(
        list(
            set(
                [
                    nmz.normalize(w) for w in codecs.open('persian-stopwords-master/persian', encoding='utf-8').read().split('\n') if w
                ]
            )
        )
    )
)
print(type(stops))


<class 'str'>


In [34]:
stops = np.array(stops.split('\n'))
stops

array(['"', '#', '(', ..., '…', '\ufeff!', '\ufeffو'], dtype='<U17')

In [40]:
word = "دوشنبه"
isin = word in stops 
isin

False

In [43]:
content = (trianDF['content'][0])

content = [i for i in content if i not in stops]
# content
content = content.split(' ')
content


# content2 = [i for i in content if i not in stops]

# print(len(content))
# print(len(content2))
# # print(content)
# print(content2)

AttributeError: 'list' object has no attribute 'split'